In [3]:
from helper import pull_records
import pandas as pd
import metadata_parser
import json


In [2]:
def get_meta(row):
    """
    get meta data from a url
    """
    meta = []
    try:
        print(f"trying row {row['title']}")
        page = metadata_parser.MetadataParser(url=row["link"])
        meta.append(page.metadata["og"])
    except Exception as e:
        print(f"oopsies on row {row['title']}")
    else:
        print(f"success row {row['title']}")

    return meta


def scrape_metadata(records, export=False):
    """
    get meta data from a list of records
    """
    #only try to get metadata for records we dont already have
    try:
        with open("clean_meta.json", "r") as file:
            clean_meta = json.load(file)
    except:
        clean_meta = []

    print('before: ', len(records))
    records = records[~records["link"].isin([meta['url'] for meta in clean_meta if meta.get('url',0) != 0])]
    bad_records = ['Why police so rarely get charged for killing unarmed black men like Stephon Clark', "America’s Most Diverse City Is Still Scarred by Its Past", "No game days. No bars. The pandemic is forcing some men to realize they need deeper friendships."]
    records = records[~records["title"].isin(bad_records)] 

    print('after: ', len(records))
    i = 0
    clean_meta_dict = []
    while i <= len(records):

        print(f"getting meta for {i} to {i + 100}")
        records_meta = records[i:i + 100]      
        meta = records_meta.apply(get_meta, axis=1)
        print(len(meta))
        clean_meta_dict = clean_meta_dict + [r[0] for r in meta.tolist() if len(r) > 0 and len(r[0]) > 0]
        i += 100
    
    # combine clean_meta_dict and clean_meta
    clean_meta_dict = clean_meta_dict + clean_meta


    #clean_meta
    if export:
        with open("clean_meta.json", "w") as fp:
            json.dump(clean_meta_dict, fp)


In [4]:
with open("clean_meta.json", "r") as file:
    clean_meta = json.load(file)

In [5]:
len(clean_meta)

238

In [10]:
page = metadata_parser.MetadataParser(url='https://www.nytimes.com/2024/06/22/well/the-vexing-problem-of-the-medium-friend.html')

/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1543: FutureWarning: `search_head_only` was not provided and defaulting to `True` Future versions will default to `False`.
  warn_future(
MetadataParser.fetch_url(https://www.nytimes.com/2024/06/22/well/the-vexing-problem-of-the-medium-friend.html)
NotParsableFetchError | https://www.nytimes.com/2024/06/22/well/the-vexing-problem-of-the-medium-friend.html | status_code: 403
NotParsableFetchError History | 0 | 403 : https://www.nytimes.com/2024/06/22/well/the-vexing-problem-of-the-medium-friend.html 


NotParsableFetchError: NotParsableFetchError: Status Code is not 200 | 403 | None

In [9]:
page.metadata

/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1619: FutureWarning: MetadataParser.metadata is deprecated in 1.0; Operate on the parsed result directly.
  warn_future(


{'og': {'url': 'https://www.nytimes.com/interactive/2020/12/17/world/asia/india-pollution-inequality.html',
  'type': 'article',
  'title': 'Who Gets to Breathe Clean Air in New Delhi? (Published 2020)',
  'image': 'https://static01.nyt.com/images/2020/12/28/world/delhi-air-promo/delhi-air-promo-facebookJumbo.jpg?year=2020&h=550&w=1050&s=7d5ac8eb518014e40c586399d167d29fcd789b118bc215b3779bc7e1b49e569e&k=ZQJBKqZ0VN',
  'image:alt': '',
  'description': 'We measured how much air pollution two children, Monu and Aamya, breathed for a day to see how much inequality makes a difference.'},
 'meta': {'charset': 'utf-8',
  'robots': 'noarchive, max-image-preview:large',
  'description': 'We measured how much air pollution two children, Monu and Aamya, breathed for a day to see how much inequality makes a difference.',
  'og:url': 'https://www.nytimes.com/interactive/2020/12/17/world/asia/india-pollution-inequality.html',
  'og:type': 'article',
  'og:title': 'Who Gets to Breathe Clean Air in N

In [7]:
records = pull_records()
scrape_metadata(records,export = True)

/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1543: FutureWarning: `search_head_only` was not provided and defaulting to `True` Future versions will default to `False`.
  warn_future(
MetadataParser.fetch_url(https://www.nytimes.com/2017/05/31/business/economy/volatile-income-economy-jobs.html)
NotParsableFetchError | https://www.nytimes.com/2017/05/31/business/economy/volatile-income-economy-jobs.html | status_code: 403
NotParsableFetchError History | 0 | 403 : https://www.nytimes.com/2017/05/31/business/economy/volatile-income-economy-jobs.html 
MetadataParser.fetch_url(https://www.nytimes.com/interactive/2018/02/28/opinion/the-unmet-promise-of-equality.html?fbclid=IwAR0jOncWvLhzZmlOB7MPzdvbIK8faPP6nqQQcEmE11-aKJtmJd_He--VfI0)


before:  361
after:  207
getting meta for 0 to 100
trying row Steady Jobs, With Pay and Hours That Are Anything But
oopsies on row Steady Jobs, With Pay and Hours That Are Anything But
trying row The Unmet Promise of Equality


/Users/austinsteinhart/anaconda3/lib/python3.10/site-packages/metadata_parser/__init__.py:1619: FutureWarning: MetadataParser.metadata is deprecated in 1.0; Operate on the parsed result directly.
  warn_future(
MetadataParser.fetch_url(https://www.citylab.com/equity/2017/09/distressed-communities/541044/?utm_source=SFFB)


success row The Unmet Promise of Equality
trying row America's Most and Least Distressed Cities


NotParsable | https://www.citylab.com/equity/2017/09/distressed-communities/541044/?utm_source=SFFB | unknown filetype, response: 
MetadataParser.fetch_url(https://www.nytimes.com/interactive/2020/09/24/climate/fires-worst-year-california-oregon-washington.html?referringSource=articleShare)
MetadataParser.fetch_url(https://www.nytimes.com/interactive/2018/10/12/us/map-of-every-building-in-the-united-states.html?fbclid=IwAR3GULdvzQ7L2TZyzz5esoi79yIyzvQgQKkW7Af4uTYhWxR-1KL35L9Nf2U)


...

NotParsableFetchError | https://www.nytimes.com/2019/12/14/us/politics/bloomberg-mayors-2020.html?smid=nytcore-ios-share | status_code: 403
NotParsableFetchError History | 0 | 403 : https://www.nytimes.com/2019/12/14/us/politics/bloomberg-mayors-2020.html?smid=nytcore-ios-share 
MetadataParser.fetch_url(https://www.nytimes.com/interactive/2017/02/27/us/politics/most-important-problem-gallup-polling-question.html?smid=fb-nytimes&smtyp=cur&_r=1&mtrref=www.facebook.com)
MetadataParser.fetch_url(http://progressive.org/magazine/letter-nephew/)


oopsies on row ‘Mayors for Mike’: How Bloomberg’s Money Built a 2020 Political Network
trying row ‘What Do You Think Is the Most Important Problem Facing This Country Today?’
success row ‘What Do You Think Is the Most Important Problem Facing This Country Today?’
trying row A Letter to My Nephew


MetadataParser.fetch_url(https://www.washingtonpost.com/graphics/2018/national/segregation-us-cities/?utm_term=.e0cf22d782f1)


success row A Letter to My Nephew
trying row America is more diverse than ever—but still segregated


KeyboardInterrupt: 